In [6]:
""" Training scheme:

1) Definition of the dataloaders for training and testing sets
2) Initialization of the weights (Gaussian, mean = 0 (?), std = sqrt(2/N))
3) Computation of the weight map    #I think this part has to be removed (the weight map is used in the main for the gt)
4) Define CrossEntopyLoss (weighted loss)
5) Optimizer (SGD with momentum = 0.99, LR?)
6) Define number of epochs, batch_size, maybe variable LR?
"""

' Training scheme:\n\n1) Definition of the dataloaders for training and testing sets\n2) Initialization of the weights (Gaussian, mean = 0 (?), std = sqrt(2/N))\n3) Computation of the weight map    #I think this part has to be removed (the weight map is used in the main for the gt)\n4) Define CrossEntopyLoss (weighted loss)\n5) Optimizer (SGD with momentum = 0.99, LR?)\n6) Define number of epochs, batch_size, maybe variable LR?\n'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import network
import functions
import data

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

from tqdm import tqdm # See progress in terminal

In [11]:
cur_dir = "/Users/pere/opt/anaconda3/envs/DL_Delft/DL_unet_master_v1"
data.download_all_data()

Extracting - DIC-C2DH-HeLa-training
Done - DIC-C2DH-HeLa-training
Extracting - DIC-C2DH-HeLa-challenge
Done - DIC-C2DH-HeLa-challenge
Extracting - PhC-C2DH-U373-training
Done - PhC-C2DH-U373-training
Extracting - PhC-C2DH-U373-challenge
Done - PhC-C2DH-U373-challenge
Done - ISBI2012-training
Done - ISBI2012-training
Done - ISBI2012-challenge


In [2]:
# Path definition:
path_images = "/Users/pere/opt/anaconda3/envs/DL_Delft/reproducibility/unet/images"
path_labels = "/Users/pere/opt/anaconda3/envs/DL_Delft/reproducibility/unet/labels"

In [4]:
# Initialization of the network
unet = network.Unet()

In [5]:
#Check for available GPUs
use_gpu = torch.cuda.is_available()

if use_gpu:
    unet = unet.cuda()

In [77]:
# Training 
num_epochs = 20
batch_size = 20

In [78]:
# Adapt this part to the actual dataset
dataset = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [79]:
data_loader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=4 # Criteria?
        )

In [95]:
len(dataset)

60000

In [91]:
train_per = 0.7
val_per = 0.3

In [92]:
print(batch_size, len(data_loader), dataset_size)

20 3000 1200000


In [93]:
[int(dataset_size * train_per), int(dataset_size * val_per)]

[840000, 360000]

In [96]:
train_set, val_set = torch.utils.data.random_split(dataset, [int(len(dataset) * train_per), int(len(dataset) * val_per)])

In [97]:
batch = next(iter(train_set))

In [98]:
images, labels = batch

In [99]:
images.shape

torch.Size([1, 28, 28])

In [76]:
preds = unet(images)

weight_map = functions.weighted_map(labels)

loss = criterion(preds, labels, weight_map)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 1 3 3, but got 3-dimensional input of size [1, 28, 28] instead